# Mundo de la aspiradora

Este notebook presenta un agente que interactúa en el mundo de la aspiradora, tal como se presenta en Russel & Norvig (2021). Una versión más compleja de este mundo puede encontrarse en <url>https://github.com/rayheberer/AI-A-Modern-Approach/tree/master/Chapter%202%20Intelligent%20Agents</url>.

### Creación del ambiente

El ambiente consiste en un conjunto de cuadrados a través de los cuales el agente puede desplazarse. Estos cuadrados se definirán como objetos que contienen un nombre y un conjunto de movimientos.

In [1]:
class Square(object):
    def __init__(self, name):
        """
        Crea un objeto square, que representa los cuadrados del ambiente.

        Argumentos
        ----------
        name : str
          Nombre del cuadrado ('A', 'B', etc.)
        """
        self.name = name
        
        # Hasta que no se coloque con cuadrados vecinos, los movimientos no llevan a ningún lado
        self.left = self
        self.right = self
        #self.up = self
        #self.down = self

En el ejemplo más simple, el ambiente se conforma por dos cuadrados nombrados 'A' y 'B':

Construiremos estos cuadrados de tal forma que el cuadrado 'A' este a la izquierda del cuadrado 'B' y, por tanto, el cuadrado 'B' a la derecha del 'A'.
Para desplazarse el agente podrá moverse a la izquierda o a la derecha. 

$$[A] [B]$$

Asimismo, uno o ambos cuadrados pueden estar sucios (lo que se indica con 1) o limpios (indicado con 0). El ambiente indicará, de manera, aleatoria qué cuadrado está o no sucio.

In [2]:
import random

class VacuumWorld(object):
    def __init__(self, dirt_init='random', move_penalty=False):
        """
        Objeto que crea el ambiente para el mundo de la aspiradora:
        Se conforma de:
        - Dos cuadrados: A y B (A a la izquierda de B)
        - Indicación de la limpieza (0) o suciedad (1) de los cuadrados.

        Argumentos
        ----------
        dirt_init : str
          Forma en que se inicializará la suciedad de los cuadrados.
        move_penalty : boolean
          Señala si se penalizará o no por el movimiento del agente en el ambiente.
        init_loc : str
          Cuadrado en que se iniciará el agente (A o B).
        """
        self.squares = []
        self.A = Square('A')
        self.B = Square('B')
        self.squares.append(self.A)
        self.squares.append(self.B)

        #Cambia la posición de A a B
        self.A.right = self.B
        #Cambia la posición de B a A
        self.B.left = self.A

        #Inicialización de suciedad
        self.dirt_init = dirt_init
        #Penalización por movimiento
        self.move_penalty = move_penalty
        
    def initialize_dirt(self):
        """
        Inicializa la suciedad de los cuadrados:

        Opciones (se indican en init)
        --------
        random : Inicializa con uno de los cuadrados sucios y otro limpio de forma aleatoria.
        dirty : Inicializa con los dos cuadrados sucios
        clean : Inicializa con los dos cuadrados limpios
        """
        if self.dirt_init=='random':
            for square in self.squares:
                if random.random() > 0.5:
                    square.dirt = 1
                else:
                    square.dirt = 0

        elif self.dirt_init=='dirty':
            for square in self.squares:
                square.dirt = 1

        elif self.dirt_init=='clean':
            for square in self.squares:
                square.dirt = 0

        else:
            for square, value in zip(self.squares, self.dirt_init):
                square.dirt = value
                    
    def initialize_agent_location(self, agent):
        """
        Inicializa la localización de un agente dado el objeto agente.

        Argumentos
        ----------
        agent : object
          Objeto agente que interecturá con el ambiente.
        """
        #Inicializa al objeto de forma aleatoria
        i = random.randint(0, len(self.squares)-1)
        agent.location = self.squares[i]

    def performance(self, action):
        """
        Determina el rendimiento del agente en base a los movimientos que realiza.

        Argumentos
        ----------
        action : str
          Action que ha realizado el agente.
        """
        move = 0
        if self.move_penalty:
            if action is not None and action != 'Clean':
                move += 1
        dirt = sum([square.dirt for square in self.squares])
        
        return len(self.squares) - dirt - move

### Creación del agente

El agente será un objeto que pueda interactuar con el ambiente. Este se compondrá de los siguientes elementos:

* Sensores: El agente tiene acceso a la información del lugar (cuadrado) en que está y si este cuadrado está sucio.
* Actuadores: El agente puede realizar las siguientes acciones:
  1. Moverse a la izquierda ('Left').
  2. Moverse a la derecha ('Right').
  3. Limpiar ('Clean').

In [3]:
class Agent(object):
    """
    Objeto agente.
    """
    def decide(self, location, dirt):
        """
        Función del agente que decidirá que hacer según cada percepción.

        Argumentos
        ----------
        location : object
          Lugar (cuadrado) en donde se encuentra el agente en el momento actual.
        dirt : {0,1}
          Indica si el lugar actual del agente está o no sucio.
        """
        if dirt:
            return 'Clean'
        elif location.name == 'A':
            return 'Right'
        elif location.name == 'B':
            return 'Left'

## Interacción del agente en el ambiente

En primer lugar, construiremos el ambiente. Señalamos que la suciedad se inicializará de manera aleatoria y que se penalizará el movimiento del agente.

In [4]:
#Creación del ambiente
env = VacuumWorld(dirt_init='random', move_penalty=True)

Ya que hemos creado el ambiente con los dos cuadrados 'A' y 'B' inicializamos la suciedad.

El ambiente entonces será un conjunto de tuplas ${(A,dirt), (B,dirt)}$, donde $dirt \in \{0,1\}$ indica si el cuadrado está o no sucio.

In [5]:
#Inicializa la suciedad
env.initialize_dirt()

print('Inicialización de estados:\n{}'.format([(s.name, s.dirt) for s in env.squares]))

Inicialización de estados:
[('A', 1), ('B', 0)]


Ahora podemos crear el agente e indicar su posición en el ambiente.

In [6]:
#Creamos el agente
agent = Agent()
#Inicializamos la localización del agente
env.initialize_agent_location(agent)

print('Posición actual del agente: {}'.format(agent.location.name))

Posición actual del agente: A


Finalmente, indicamos como va a interactuar el agente con el ambiente. En este caso, el agente percibirá y realizará acciones hasta que los dos cuadros estén limpios.

La siguiente tabla resume las función del agente que hemos indicado:

| Percepción | Acción |
| :- | -: |
| (A,0) | Right |
| (B,0) | Left |
| (A,1) | Clean |
| (B,1) | Clean |

In [7]:
while env.squares[0].dirt == 1 or env.squares[1].dirt == 1:
  #Realiza acción en base a lugar y suciedad
  location = agent.location
  dirt = agent.location.dirt
  action = agent.decide(location,dirt)
  
  print('Ubicación del agente: {}\n¿Está sucio?: {}\nAcción a realizar: {}\n'.format(location.name, dirt, action))
  
  #(A,1) -> Clean ;  (B,1)-> Clean
  if action == 'Clean':
    agent.location.dirt = 0
  #(A,0) -> Right
  if action == 'Right':
    agent.location = env.A.right
  #(B,0) -> Left
  if action == 'Left':
    agent.location = env.B.left

Ubicación del agente: A
¿Está sucio?: 1
Acción a realizar: Clean



In [8]:
print('Estados después del agente:\n{}'.format([(s.name, s.dirt) for s in env.squares]))

Estados después del agente:
[('A', 0), ('B', 0)]
